# 1 – Data Prep (Final)

Dieses Notebook erzeugt für eine frei wählbare `EXP_ID` die Labels
und den Trainingsdatensatz für das Zwei-Stufen-XGBoost-Modell.

Es ruft nur die Python-Pipelines auf und enthält bewusst **kein**
Modell-Training oder umfangreiche Visualisierungen.


In [1]:
import sys
from pathlib import Path
import os

cwd = Path.cwd()
project_root = cwd
while not (project_root / 'src').is_dir():
    if project_root.parent == project_root:
        raise RuntimeError("Projektwurzel mit 'src' nicht gefunden.")
    project_root = project_root.parent

print('Erkannte Projektwurzel:', project_root)
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

os.chdir(project_root)
print('Arbeitsverzeichnis gesetzt auf:', Path.cwd())


Erkannte Projektwurzel: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project
Arbeitsverzeichnis gesetzt auf: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project


In [2]:
# Experiment-Konfiguration – bitte EXP_ID explizit setzen, bevor du das Notebook ausführst.
#
# Dieses Notebook ist aktuell auf das erste hv-Experiment eingestellt,
# das die neue Label-Definition (Treffer irgendwo im Horizont) nutzt.

EXP_ID = 'hv5_h4_thr0p4pct_hit_2'  # <- hier setzen
assert EXP_ID != 'CHANGE_ME', 'Bitte EXP_ID oben setzen, bevor du weiterläufst.'

# Label-Parameter – hv1: 4 Tage, ±0.3 %, nicht strikt,
# max. 1 % gegenläufige Bewegung erlaubt, Treffer reicht,
# sobald die Schwelle innerhalb des Horizonts einmal erreicht wird.
LABEL_PARAMS = dict(
    horizon_days=4,
    up_threshold=0.004,
    down_threshold=-0.004,
    strict_monotonic=False,
    max_adverse_move_pct=0.01,
    # Neue Option: Wenn True, wird ein Label bereits vergeben,
    # sobald die Schwelle irgendwo innerhalb des Horizonts
    # [t, t+horizon_days] einmal erreicht/ueberschritten wird.
    # Standard=False -> nur Endkurs am Horizonttag entscheidet.
    hit_within_horizon=True,
)


In [3]:
import json
from pathlib import Path

from src.utils.io import DATA_PROCESSED
from src.data.label_eurusd import label_eurusd
from src.data.build_training_set import build_training_dataframe

print('EXP_ID:', EXP_ID)
print('LABEL_PARAMS:', LABEL_PARAMS)

# 1) Config-JSON speichern
exp_meta_dir = DATA_PROCESSED / 'experiments'
exp_meta_dir.mkdir(parents=True, exist_ok=True)
exp_config = {
    'exp_id': EXP_ID,
    'label_params': LABEL_PARAMS,
}
exp_config_path = exp_meta_dir / f'{EXP_ID}_config.json'
with exp_config_path.open('w', encoding='utf-8') as f:
    json.dump(exp_config, f, indent=2)
print('[ok] Experiment-Konfiguration gespeichert unter:', exp_config_path)

# 2) Labels berechnen
labels = label_eurusd(**LABEL_PARAMS)
fx_dir = DATA_PROCESSED / 'fx'
fx_dir.mkdir(parents=True, exist_ok=True)
labels_path_exp = fx_dir / f'eurusd_labels__{EXP_ID}.csv'
labels.to_csv(labels_path_exp)
labels_path_latest = fx_dir / 'eurusd_labels.csv'
labels.to_csv(labels_path_latest)
print('[ok] Labels gespeichert als:')
print('   ', labels_path_exp)
print('   ', labels_path_latest)

# 3) Trainingsdatensatz bauen
merged = build_training_dataframe(exp_id=EXP_ID)
ds_dir = DATA_PROCESSED / 'datasets'
ds_dir.mkdir(parents=True, exist_ok=True)
train_path_exp = ds_dir / f'eurusd_news_training__{EXP_ID}.csv'
merged.to_csv(train_path_exp, index=False)
train_path_latest = ds_dir / 'eurusd_news_training.csv'
merged.to_csv(train_path_latest, index=False)
print('[ok] Trainingsdatensatz gespeichert als:')
print('   ', train_path_exp)
print('   ', train_path_latest)


EXP_ID: hv5_h4_thr0p4pct_hit_2
LABEL_PARAMS: {'horizon_days': 4, 'up_threshold': 0.004, 'down_threshold': -0.004, 'strict_monotonic': False, 'max_adverse_move_pct': 0.01, 'hit_within_horizon': True}
[ok] Experiment-Konfiguration gespeichert unter: data/processed/experiments/hv5_h4_thr0p4pct_hit_2_config.json
[ok] Labels gespeichert als:
    data/processed/fx/eurusd_labels__hv5_h4_thr0p4pct_hit_2.csv
    data/processed/fx/eurusd_labels.csv
[ok] Trainingsdatensatz gespeichert als:
    data/processed/datasets/eurusd_news_training__hv5_h4_thr0p4pct_hit_2.csv
    data/processed/datasets/eurusd_news_training.csv
